# Iteration 6: add in a count of occupancy stage 2

## Plain English Summary

We modify ACU to include a count of the concurrent number of patients on the ward. This modifies the acute treatment functions in the ASU class.

We carry over the `Experiment` class and the lognormal parameter function from iteration 5.

> Testing Note: Now that we have a measure of occupancy we can modify the extreme value tests looking at infinite acute treatment times.

## Notebook imports

In [1]:
from sim_tools.distributions import Lognormal

## LLM Prompt

::::{admonition} LLM Prompt
:::{include} ../04_prompts_stage1/inputs/06_iteration.md
:::
::::

## Date and time of input to LLM

> 0830 10th July 2024

## Observations on the LLM Output

* The code appears to have been modified correctly with comments highlighting the changes

## Sources reported by the LLM

None reported

## LLM Output

Certainly! I'll modify the AcuteStrokeUnit class as requested. Here's the updated AcuteStrokeUnit class with the new occupancy member variable and the necessary modifications to track it:

In [4]:
import simpy
import numpy as np
import math

class Experiment:
    def __init__(self, params=None):
        default_params = {
            'run_length': 5 * 365,
            'patient_types': {
                'Stroke': {
                    'interarrival_time': 1.2,
                    'post_asu_probabilities': [0.24, 0.13, 0.63],
                    'los_params': {'Rehab': (7.4, 8.6), 'ESD': (4.6, 4.8), 'Other': (7.0, 8.7)}
                },
                'TIA': {
                    'interarrival_time': 9.3,
                    'post_asu_probabilities': [0.01, 0.01, 0.98],
                    'los_params': (1.8, 5.0)
                },
                'Complex Neurological': {
                    'interarrival_time': 3.6,
                    'post_asu_probabilities': [0.11, 0.05, 0.84],
                    'los_params': (4.0, 5.0)
                },
                'Other': {
                    'interarrival_time': 3.2,
                    'post_asu_probabilities': [0.05, 0.10, 0.85],
                    'los_params': (3.8, 5.2)
                }
            }
        }
        
        if params is None:
            self.params = default_params
        else:
            self.params = self.merge_params(default_params, params)

    def merge_params(self, default, new):
        merged = default.copy()
        for key, value in new.items():
            if isinstance(value, dict) and key in merged:
                merged[key] = self.merge_params(merged[key], value)
            else:
                merged[key] = value
        return merged

class PatientType:
    def __init__(self, name, interarrival_time, post_asu_probabilities, los_params):
        self.name = name
        self.interarrival_time = interarrival_time
        self.count = 0
        self.rng = np.random.default_rng()
        self.post_asu_probabilities = post_asu_probabilities
        self.los_params = los_params

    def generate_interarrival_time(self):
        return self.rng.exponential(self.interarrival_time)

    def sample_post_asu_destination(self):
        return self.rng.choice(['Rehab', 'ESD', 'Other'], p=self.post_asu_probabilities)

    def sample_length_of_stay(self, destination=None):
        if self.name == 'Stroke':
            mean, std = self.los_params[destination]
        else:
            mean, std = self.los_params

        normal_mean = math.log(mean**2 / math.sqrt(std**2 + mean**2))
        normal_std = math.sqrt(math.log(1 + (std**2 / mean**2)))

        return self.rng.lognormal(normal_mean, normal_std)

class AcuteStrokeUnit:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.run_length = experiment.params['run_length']
        self.total_arrivals = 0
        self.occupancy = 0  # New member variable to track occupancy
        
        self.patient_types = {
            name: PatientType(name, 
                              params['interarrival_time'], 
                              params['post_asu_probabilities'], 
                              params['los_params'])
            for name, params in experiment.params['patient_types'].items()
        }

    def run(self):
        for patient_type in self.patient_types.values():
            self.env.process(self.patient_generator(patient_type))
        self.env.run(until=self.run_length)

    def patient_generator(self, patient_type):
        while True:
            interarrival_time = patient_type.generate_interarrival_time()
            yield self.env.timeout(interarrival_time)
            
            self.total_arrivals += 1
            patient_type.count += 1
            patient_id = self.total_arrivals - 1

            post_asu_destination = patient_type.sample_post_asu_destination()

            print(f"Time {self.env.now:.2f}: Patient {patient_id} ({patient_type.name}) arrived")
            print(f"  Total arrivals: {self.total_arrivals}")
            print(f"  {patient_type.name} arrivals: {patient_type.count}")
            print(f"  Post-ASU destination: {post_asu_destination}")
            print(f"  Next {patient_type.name} arrival in {interarrival_time:.2f} days")

            self.occupancy += 1  # Increment occupancy when patient arrives
            print(f"  Current occupancy: {self.occupancy}")

            self.env.process(self.acute_treatment(patient_type, patient_id, post_asu_destination))

    def acute_treatment(self, patient_type, patient_id, post_asu_destination):
        if patient_type.name == 'Stroke':
            yield from self.stroke_acute_treatment(patient_type, patient_id, post_asu_destination)
        elif patient_type.name == 'TIA':
            yield from self.tia_acute_treatment(patient_type, patient_id)
        elif patient_type.name == 'Complex Neurological':
            yield from self.complex_neurological_acute_treatment(patient_type, patient_id)
        else:  # Other
            yield from self.other_acute_treatment(patient_type, patient_id)

        self.occupancy -= 1  # Decrement occupancy when patient leaves
        print(f"Time {self.env.now:.2f}: Patient {patient_id} ({patient_type.name}) left ASU")
        print(f"  Current occupancy: {self.occupancy}")

    def stroke_acute_treatment(self, patient_type, patient_id, post_asu_destination):
        los = patient_type.sample_length_of_stay(post_asu_destination)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Stroke) starting acute treatment")
        print(f"  Length of stay: {los:.2f} days")
        yield self.env.timeout(los)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Stroke) finished acute treatment")

    def tia_acute_treatment(self, patient_type, patient_id):
        los = patient_type.sample_length_of_stay()
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (TIA) starting acute treatment")
        print(f"  Length of stay: {los:.2f} days")
        yield self.env.timeout(los)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (TIA) finished acute treatment")

    def complex_neurological_acute_treatment(self, patient_type, patient_id):
        los = patient_type.sample_length_of_stay()
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Complex Neurological) starting acute treatment")
        print(f"  Length of stay: {los:.2f} days")
        yield self.env.timeout(los)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Complex Neurological) finished acute treatment")

    def other_acute_treatment(self, patient_type, patient_id):
        los = patient_type.sample_length_of_stay()
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Other) starting acute treatment")
        print(f"  Length of stay: {los:.2f} days")
        yield self.env.timeout(los)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Other) finished acute treatment")

def main(params=None):
    experiment = Experiment(params)
    env = simpy.Environment()
    model = AcuteStrokeUnit(env, experiment)
    model.run()

    print("\nSimulation completed.")
    print(f"Total simulation time: {model.env.now:.2f} days")
    print(f"Total patient arrivals: {model.total_arrivals}")
    for patient_type in model.patient_types.values():
        print(f"Total {patient_type.name} arrivals: {patient_type.count}")

if __name__ == "__main__":
    main()


Time 0.05: Patient 0 (Stroke) arrived
  Total arrivals: 1
  Stroke arrivals: 1
  Post-ASU destination: Other
  Next Stroke arrival in 0.05 days
  Current occupancy: 1
Time 0.05: Patient 0 (Stroke) starting acute treatment
  Length of stay: 2.53 days
Time 0.71: Patient 1 (TIA) arrived
  Total arrivals: 2
  TIA arrivals: 1
  Post-ASU destination: Other
  Next TIA arrival in 0.71 days
  Current occupancy: 2
Time 0.71: Patient 1 (TIA) starting acute treatment
  Length of stay: 1.39 days
Time 1.12: Patient 2 (TIA) arrived
  Total arrivals: 3
  TIA arrivals: 2
  Post-ASU destination: Other
  Next TIA arrival in 0.41 days
  Current occupancy: 3
Time 1.12: Patient 2 (TIA) starting acute treatment
  Length of stay: 0.08 days
Time 1.20: Patient 2 (TIA) finished acute treatment
Time 1.20: Patient 2 (TIA) left ASU
  Current occupancy: 2
Time 1.42: Patient 3 (Complex Neurological) arrived
  Total arrivals: 4
  Complex Neurological arrivals: 1
  Post-ASU destination: Rehab
  Next Complex Neurologica

## Testing
* **Testing lognormal function**: Test that lognomal function correctly calculates the moments of the underlying normal dist.
    * Expected result: The LLM generated function and `sim-tools.distributions.Lognormal` perform the same conversion.
    * Actual results (PASS):  The functions generate the same values.
* **Extreme value test 1**: TIA, Complex Neuro, Other, have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Stroke"
    * Actual result (PASS): The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: No patients arrive to the model.
    * Actual result (PASS): No patients arrive to the model.
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 10 simulated time periods
    * Actual result (PASS): The model runs no longer than 10 simulated time periods
* **Extreme value test 3**: All patient types have have their mean length of stay time set to $M$ a very large number
    * Expected result: No patients depart the model (only arrivals seen in log)
    * Actual Results (PASS): No patients depart the model.
* **Extreme value test 4**: All patient types apart from stroke->rehab have have their mean length of stay time set to $M$ a very large number
    * Expected result: Only stroke patients depart the model (seen in log)
    * Actual Results (PASS): Only stroke patient depart the model.

### Lognormal test

Test that lognomal function correctly calculates the moments of the underlying normal dist.

**Result: PASS**

In [5]:
def test_lognormal_moments(mean, std):
    '''
    Test that lognomal function correctly calculates 
    the moments of the underlying normal dist.

    Params:
    ------
    mean: float
        mean of the lognormal distribution

    std: float
        st dev of the lognormal distribution

    Returns:
    -------
    bool
    '''
   
    # Lognormal class from sim-tools.
    expected_moments = Lognormal(mean, std)
    print(expected_moments.mu, expected_moments.sigma)

    # Convert lognormal parameters from llm
    normal_mean = math.log(mean**2 / math.sqrt(std**2 + mean**2))
    normal_std = math.sqrt(math.log(1 + (std**2 / mean**2)))
    print(normal_mean, normal_std)

    return (normal_mean, normal_std) == (expected_moments.mu, expected_moments.sigma)

In [6]:
test_lognormal_moments(128.79, 267.51)

4.022977459259615 1.2924439729790718
4.022977459259615 1.2924439729790718


True

### Extreme value test 1

The model uses hard coded parameters.  The most simple way to modify the code for this test was to change hard coded parameters for $M$

**Result: PASS**

In [7]:
M = 10_000_000

In [8]:
def test_1(large_number):
    '''
    TIA, Complex Neuro, Other, have their inter-arrival 
    time set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model 
    is "Stroke". This is seen in the event log

    Params:
    -------
    large_number: int
        M a very large number 

    Returns:
    --------
    int: the number of patients that arrived to the model.
    '''

    def main(arrival_rates=None, run_length=None):
        custom_params = {}
        
        if arrival_rates:
            custom_params['patient_types'] = arrival_rates
        
        if run_length:
            custom_params['run_length'] = run_length
    
        # Create an Experiment instance with custom parameters
        experiment = Experiment(custom_params)
    
        # Run the simulation with the custom experiment
        env = simpy.Environment()
        model = AcuteStrokeUnit(env, experiment)
        model.run()
    
        # Print results to check
        print("\nSimulation completed.")
        print(f"Total simulation time: {model.env.now:.2f} days")
        print(f"Total patient arrivals: {model.total_arrivals}")
        for patient_type in model.patient_types.values():
            print(f"Total {patient_type.name} arrivals: {patient_type.count}")
    
    if __name__ == "__main__":
        large_number = M
        
        asu_arrival_rates = {
            'Stroke': {'interarrival_time': 1.2},
            'TIA': {'interarrival_time': large_number},
            'Complex Neurological': {'interarrival_time': large_number},
            'Other': {'interarrival_time': large_number}
        }
    
    main(asu_arrival_rates, run_length=2*365)  # Run for 2 years with custom parameters


In [9]:
test_1(M)

Time 0.46: Patient 0 (Stroke) arrived
  Total arrivals: 1
  Stroke arrivals: 1
  Post-ASU destination: Other
  Next Stroke arrival in 0.46 days
  Current occupancy: 1
Time 0.46: Patient 0 (Stroke) starting acute treatment
  Length of stay: 0.94 days
Time 1.40: Patient 0 (Stroke) finished acute treatment
Time 1.40: Patient 0 (Stroke) left ASU
  Current occupancy: 0
Time 2.26: Patient 1 (Stroke) arrived
  Total arrivals: 2
  Stroke arrivals: 2
  Post-ASU destination: Rehab
  Next Stroke arrival in 1.81 days
  Current occupancy: 1
Time 2.26: Patient 1 (Stroke) starting acute treatment
  Length of stay: 15.91 days
Time 2.56: Patient 2 (Stroke) arrived
  Total arrivals: 3
  Stroke arrivals: 3
  Post-ASU destination: Other
  Next Stroke arrival in 0.30 days
  Current occupancy: 2
Time 2.56: Patient 2 (Stroke) starting acute treatment
  Length of stay: 30.16 days
Time 5.63: Patient 3 (Stroke) arrived
  Total arrivals: 4
  Stroke arrivals: 4
  Post-ASU destination: Rehab
  Next Stroke arrival 

### Extreme value test 2

* An experiment is setup so that all patient types have a mean inter-arrival rate of $M$

**Result: PASS**

In [10]:
def test_2(large_number):
    '''
    All patient types have have their inter-arrival time is 
    set to $M$ a very large number
    
    Expected result: No patients arrive to the model.
    
    Params:
    -------
    large_number: int
        M a very large number 

    Returns:
    --------
    int: the number of patients that arrived to the model.
    '''

    def main(arrival_rates=None, run_length=None):
        custom_params = {}
        
        if arrival_rates:
            custom_params['patient_types'] = arrival_rates
        
        if run_length:
            custom_params['run_length'] = run_length
    
        # Create an Experiment instance with custom parameters
        experiment = Experiment(custom_params)
    
        # Run the simulation with the custom experiment
        env = simpy.Environment()
        model = AcuteStrokeUnit(env, experiment)
        model.run()
    
        # Print results to check
        print("\nSimulation completed.")
        print(f"Total simulation time: {model.env.now:.2f} days")
        print(f"Total patient arrivals: {model.total_arrivals}")
        for patient_type in model.patient_types.values():
            print(f"Total {patient_type.name} arrivals: {patient_type.count}")
    
    if __name__ == "__main__":
        large_number = M
        
        asu_arrival_rates = {
            'Stroke': {'interarrival_time': large_number},
            'TIA': {'interarrival_time': large_number},
            'Complex Neurological': {'interarrival_time': large_number},
            'Other': {'interarrival_time': large_number}
        }
    
    main(asu_arrival_rates, run_length=2*365)  # Run for 2 years with custom parameters


In [11]:
test_2(M)


Simulation completed.
Total simulation time: 730.00 days
Total patient arrivals: 0
Total Stroke arrivals: 0
Total TIA arrivals: 0
Total Complex Neurological arrivals: 0
Total Other arrivals: 0


### Different run length test

The model has again been modified to use the original code and parameters generated by the LLM.

In [12]:
TEN_DAYS = 10

In [13]:
def test_3(run_length):
    '''
    Test that the run length of the model can be modified.
    
    Expected result: The model should runs no longer than the new run length.

    Params:
    ------
    run_length: float
        Run length of the model in days.

    Returns:
    -------
    float
        Simulation end time.
    '''
    def main(run_length=None):
        custom_params = {}
        
        if run_length:
            custom_params['run_length'] = run_length
    
        # Create an Experiment instance with custom parameters
        experiment = Experiment(custom_params)
    
        # Run the simulation with the custom experiment
        env = simpy.Environment()
        model = AcuteStrokeUnit(env, experiment)
        model.run()
    
        # Print results to check
        print("\nSimulation completed.")
        print(f"Total simulation time: {model.env.now:.2f} days")
        print(f"Total patient arrivals: {model.total_arrivals}")
        for patient_type in model.patient_types.values():
            print(f"Total {patient_type.name} arrivals: {patient_type.count}")
    if __name__ == "__main__":
        main(run_length=run_length)


In [14]:
test_3(TEN_DAYS)

Time 0.09: Patient 0 (Stroke) arrived
  Total arrivals: 1
  Stroke arrivals: 1
  Post-ASU destination: Other
  Next Stroke arrival in 0.09 days
  Current occupancy: 1
Time 0.09: Patient 0 (Stroke) starting acute treatment
  Length of stay: 1.68 days
Time 1.10: Patient 1 (Complex Neurological) arrived
  Total arrivals: 2
  Complex Neurological arrivals: 1
  Post-ASU destination: Other
  Next Complex Neurological arrival in 1.10 days
  Current occupancy: 2
Time 1.10: Patient 1 (Complex Neurological) starting acute treatment
  Length of stay: 2.20 days
Time 1.61: Patient 2 (Stroke) arrived
  Total arrivals: 3
  Stroke arrivals: 2
  Post-ASU destination: Rehab
  Next Stroke arrival in 1.52 days
  Current occupancy: 3
Time 1.61: Patient 2 (Stroke) starting acute treatment
  Length of stay: 2.40 days
Time 1.76: Patient 0 (Stroke) finished acute treatment
Time 1.76: Patient 0 (Stroke) left ASU
  Current occupancy: 2
Time 2.17: Patient 3 (Other) arrived
  Total arrivals: 4
  Other arrivals: 1


### Extreme value test 3

* An experiment is setup so that all patient types have a mean length of stay on the ASU of $M$

**Result: PASS**

In [38]:
def test_4(large_number):
    '''
    All patient types have have their mean length 
    of stay time set to $M$ a very large number
    
    Expected result: No patients depart the model 
    (only arrivals seen in log)

    Params:
    -------
    large_number: int
        M a very large number 
    '''
    def main(los_params=None, run_length=None):
        custom_params = {'patient_types': {}}
        
        if los_params:
            for patient_type, los in los_params.items():
                if patient_type not in custom_params['patient_types']:
                    custom_params['patient_types'][patient_type] = {}
                custom_params['patient_types'][patient_type]['los_params'] = los
        
        if run_length:
            custom_params['run_length'] = run_length
    
        # Create an Experiment instance with custom parameters
        experiment = Experiment(custom_params)
    
        # Run the simulation with the custom experiment
        env = simpy.Environment()
        model = AcuteStrokeUnit(env, experiment)
        model.run()
    
        # Print results
        print("\nSimulation completed.")
        print(f"Total simulation time: {model.env.now:.2f} days")
        print(f"Total patient arrivals: {model.total_arrivals}")
        print(f"Total occupancy: {model.occupancy}")
        for patient_type in model.patient_types.values():
            print(f"Total {patient_type.name} arrivals: {patient_type.count}")

        return model  # Return the model so we can access its attributes

    # New length of stay parameters - stdev is default
    new_los_params = {
        'Stroke': {'Rehab': (large_number, 8.6), 'ESD': (large_number, 4.8), 'Other': (large_number, 8.7)},
        'TIA': (large_number, 5.0),
        'Complex Neurological': (large_number, 5.0),
        'Other': (large_number, 5.2)
    }

    # Run the simulation with new parameters
    model = main(los_params=new_los_params, run_length=2*365)

    # Check if the total number of arrivals is greater than the final occupancy
    total_arrivals = model.total_arrivals
    final_occupancy = model.occupancy

    return total_arrivals == final_occupancy


In [39]:
result = test_4(M)

Time 0.68: Patient 0 (Other) arrived
  Total arrivals: 1
  Other arrivals: 1
  Post-ASU destination: Rehab
  Next Other arrival in 0.68 days
  Current occupancy: 1
Time 0.68: Patient 0 (Other) starting acute treatment
  Length of stay: 10000011.36 days
Time 1.41: Patient 1 (TIA) arrived
  Total arrivals: 2
  TIA arrivals: 1
  Post-ASU destination: Other
  Next TIA arrival in 1.41 days
  Current occupancy: 2
Time 1.41: Patient 1 (TIA) starting acute treatment
  Length of stay: 9999996.41 days
Time 1.47: Patient 2 (Complex Neurological) arrived
  Total arrivals: 3
  Complex Neurological arrivals: 1
  Post-ASU destination: Other
  Next Complex Neurological arrival in 1.47 days
  Current occupancy: 3
Time 1.47: Patient 2 (Complex Neurological) starting acute treatment
  Length of stay: 10000001.66 days
Time 1.51: Patient 3 (Stroke) arrived
  Total arrivals: 4
  Stroke arrivals: 1
  Post-ASU destination: Rehab
  Next Stroke arrival in 1.51 days
  Current occupancy: 4
Time 1.51: Patient 3 (S

In [40]:
result

True

### Extreme value test 4

* An experiment is setup so that all patient types other than stroke->rehab have a mean length of stay on the ASU of $M$

**Result: PASS**

In [41]:
def test_5(large_number):
    '''
    All patient types apart from stroke-rehab patients
    have have their mean length 
    of stay time set to $M$ a very large number
    
    Expected result: Only stroke patients depart the
    model.
    (only arrivals seen in log)

    Params:
    -------
    large_number: int
        M a very large number 
    '''
    
    def main(los_params=None, run_length=None):
        custom_params = {'patient_types': {}}
        
        if los_params:
            for patient_type, los in los_params.items():
                if patient_type not in custom_params['patient_types']:
                    custom_params['patient_types'][patient_type] = {}
                custom_params['patient_types'][patient_type]['los_params'] = los
        
        if run_length:
            custom_params['run_length'] = run_length
    
        # Create an Experiment instance with custom parameters
        experiment = Experiment(custom_params)
    
        # Run the simulation with the custom experiment
        env = simpy.Environment()
        model = AcuteStrokeUnit(env, experiment)
        model.run()
    
        # Print results
        print("\nSimulation completed.")
        print(f"Total simulation time: {model.env.now:.2f} days")
        print(f"Total patient arrivals: {model.total_arrivals}")
        print(f"Total occupancy: {model.occupancy}")
        for patient_type in model.patient_types.values():
            print(f"Total {patient_type.name} arrivals: {patient_type.count}")

        return model  # Return the model so we can access its attributes

    # New length of stay parameters - stdev is default
    new_los_params = {
        'Stroke': {'Rehab': (7.4, 8.6), 'ESD': (large_number, 4.8), 'Other': (large_number, 8.7)},
        'TIA': (large_number, 5.0),
        'Complex Neurological': (large_number, 5.0),
        'Other': (large_number, 5.2)
    }

    # Run the simulation with new parameters
    model = main(los_params=new_los_params, run_length=2*365)

    # Check if the total number of arrivals is greater than the final occupancy
    total_arrivals = model.total_arrivals
    final_occupancy = model.occupancy

    return total_arrivals > final_occupancy


In [42]:
results = test_5(M)

Time 0.63: Patient 0 (Stroke) arrived
  Total arrivals: 1
  Stroke arrivals: 1
  Post-ASU destination: Other
  Next Stroke arrival in 0.63 days
  Current occupancy: 1
Time 0.63: Patient 0 (Stroke) starting acute treatment
  Length of stay: 10000016.27 days
Time 1.42: Patient 1 (Stroke) arrived
  Total arrivals: 2
  Stroke arrivals: 2
  Post-ASU destination: Rehab
  Next Stroke arrival in 0.78 days
  Current occupancy: 2
Time 1.42: Patient 1 (Stroke) starting acute treatment
  Length of stay: 2.61 days
Time 1.58: Patient 2 (Stroke) arrived
  Total arrivals: 3
  Stroke arrivals: 3
  Post-ASU destination: Other
  Next Stroke arrival in 0.16 days
  Current occupancy: 3
Time 1.58: Patient 2 (Stroke) starting acute treatment
  Length of stay: 9999988.35 days
Time 1.95: Patient 3 (Stroke) arrived
  Total arrivals: 4
  Stroke arrivals: 4
  Post-ASU destination: Other
  Next Stroke arrival in 0.37 days
  Current occupancy: 4
Time 1.95: Patient 3 (Stroke) starting acute treatment
  Length of sta

In [43]:
results

True